<a href="https://colab.research.google.com/github/Bladefinger/Computer-Vision/blob/main/OCR_with_Tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Optical Character Recognition (OCR)**


                            OCR
                             |
                      ------- ---------
                      |               | 
                Online Text      Offline Text
                                      |
                               ------- ---------
                               |               | 
                            Typed Text      Handwritten Text

Dans ce notebook ous allons commencer par faire un premier cas d'utilisations de la librairie pour l'extraction des caractères saisies (cas d'une facture imprimé). Dans un second second temps, nous allons essayer d'extraire les informations manuscrites d'une image. Nous allons Terminer par présenter les avantages et les limites de Tesseract. 


---

#####**Brell SANWOUO**




In [ ]:
!pip install asrtoolkit

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
!pip install pytesseract

In [ ]:
!pip install google-cloud-vision

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install deskew

In [ ]:
import asrtoolkit
from asrtoolkit import wer, cer
import cv2
import io
import json
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from PIL import Image
import pytesseract
import sys
from skimage import io as ioski
from skimage.color import rgb2gray
from skimage.transform import rotate, resize
import os
from deskew import determine_skew
from google.cloud import vision
from google.cloud.vision_v1 import types
from dotenv import load_dotenv, find_dotenv
load_dotenv()

####**Cas 1**: extration des **informations** d'une **facture**

In [ ]:
sample_image ="/content/test_images.png"
output_processed = "/content/" + "output_processed.png"
output_deskewed = "/content/" + "output_deskewed.png"
output_no_deskewed = "/content/" + "output_no_deskewed.png"

In [ ]:
outputimage = Image.open('/content/Facture.jpg')
print(outputimage)
outputimage

In [ ]:
print(pytesseract.image_to_string(outputimage))

In [ ]:
def RemoveEmptyLines(entree):
    tab = entree.strip()
    tableausansvide = [ x for x in tab.splitlines() if x!='' ]
    res = ''
    for i in range(0, len(tableausansvide)):
        res = res + tableausansvide[i] + '\n'
    return res
 
def getTextBetween(mainString, startWord, endWord):
    start = mainString.find(startWord) + len(startWord)
    end = mainString.find(endWord)
    return RemoveEmptyLines(mainString[start:end])
 
def getPosElement(po):
    element = {}
    element['quantite'] = po[0:po.find (' ')].strip()
    po = po[po.find (' '):len(po)]
    element['prixtotht'] = po[po.rfind (' '):len(po)].strip()
    po = po[0:po.rfind (' ')]
    element['prixunitht'] = po[po.rfind (' '):len(po)].strip()
    po = po[0:po.rfind (' ')]
    element['decription'] = po.strip()
    return element

In [ ]:
output = {}    
resultat = pytesseract.image_to_string(outputimage)
output["Adresse"] = getTextBetween(resultat, 'www.blueprism.com/fr', 'Référence').strip()
output["Reference"] = getTextBetween(resultat, 'Référence: ', 'Date: ').strip()
output["DateFacture"] = getTextBetween(resultat, 'Date: ', 'Client: ').strip()
output["CodeClient"] = getTextBetween(resultat, 'Client: ', 'Intitulé: ').strip()
 
# Récupération des lignes de PO
pos = getTextBetween(resultat, 'Prix total HT', 'Total HT ')
tabPOs = pos.splitlines()
print ('Nombre de PO: ' + str(len(tabPOs)))
output["NbPo"] = len(tabPOs)
pos = []
for i in range(0, len(tabPOs)):
    pos.append(getPosElement(tabPOs[i]))
output['po'] = pos
output["totalht"] = getTextBetween(resultat, 'Total HT ', 'TVA (20%) ').strip()
output["tva"] = getTextBetween(resultat, 'TVA (20%) ', 'Total TTC (en euros) ').strip()
output["total"] = getTextBetween(resultat, 'Total TTC (en euros) ', 'En votre aimable réglement,').strip()

In [ ]:
print (output)

In [ ]:
outputimage = Image.open('/content/test_images.png')
print(outputimage)
outputimage

In [ ]:
image = ioski.imread('/content/test_images.png')
grayscale = rgb2gray(image)
angle = determine_skew(grayscale)
print(angle)
rotated = rotate(image, angle, resize=True) * 255
print(type(rotated))
ioski.imsave("output_deskewed.jpg", rotated.astype(np.uint8))

In [ ]:
Image.open(output_deskewed)

In [ ]:
def pre_process_image(image):
    """Cette fonction va pré-traiter une image avec CV2 & deskew"""
    img = cv2.imread(image)
    img = cv2.resize(img, None, fx=.3, fy=.3) #redimensionner en utilisant les pourcentages
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #changer la couleur de l'imager de BGR à RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #formater l'image en niveau de gris
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11) # retirer l'arrière plan
    return img

In [ ]:
#Fonction pour redresser l'image
def img_process(image):
    """This function will deskew"""
    cv2output = io.imread(image)
    angle = determine_skew(cv2output)
    print("angle rotation of image",angle)
    rotated = rotate(cv2output, angle, resize=True) * 255
    return rotated

In [ ]:
# Utilisation de la fonction pre_process_image  pour sauvegareder l'image prétraiter
processed_img = pre_process_image(output_deskewed)
cv2.imwrite("output_processed.png",processed_img)
processed_img = pre_process_image(sample_image)
cv2.imwrite("output_no_deskewing.png",processed_img)

In [ ]:
sample_image ="/content/test_images.png"
output_processed = "/content/" + "output_processed.png"
output_deskewed = "/content/" + "output_deskewed.png"
output_no_deskewed = "/content/" + "output_no_deskewed.png"


In [ ]:
phrase_corect = "Delicious substitute for tuna in your favorite recipes. Perfect for creative hors d'oeuvres and snacks. Great as a salad topper"

In [ ]:
#pytesseract sans prétraitement
hypothesis0 = pytesseract.image_to_string(sample_image)
print("CER :" cer(phrase_corect, hypothesis0))
print("WER :" wer(sample_image, hypothesis0))
print("Taux :" fuzz.ratio(phrase_corect, hypothesis0))
print(hypothesis0)

In [ ]:
#pytesseract avec l'image redréssé en niveau de gris
hypothesis1 = pytesseract.image_to_string(output_deskewed)
print("CER :" cer(phrase_corect, hypothesis1))
print("WER :" wer(phrase_corect, hypothesis1))
print("Taux :" fuzz.ratio(phrase_corect, hypothesis1))
print(hypothesis1)

In [ ]:
#pytesseract avec l'image en niveau de gris non redréssé
hypothesis2 = pytesseract.image_to_string(output_no_deskewing)
print("CER :" cer(phrase_corect, hypothesis2))
print("WER :" wer(phrase_corect, hypothesis2))
print("Taux :" fuzz.ratio(phrase_corect, hypothesis2))
print(hypothesis2)

In [ ]:
#pytesseract sur l'image prétraité en RGB redréssé
hypothesis3 = pytesseract.image_to_string(output_processed)
print("CER :" cer(phrase_corect, hypothesis3))
print("WER :" wer(phrase_corect, hypothesis3))
print("Taux :" fuzz.ratio(phrase_corect, hypothesis3))
print(hypothesis3)


#**Conclusion**
EN conclusion, Tesseract permet bel et bien d'extraire des informations textuelles sur une image ou sur un pdf. mais tesseract n'est pas parfait il présente quelques limites.
Voici quelques limitations de Tesseract :

1. Malheureusement Tesseract n’est pas capable de reconnaître 
correctement l’écriture manuscrite.
2. Tesseract ne fonctionne pas vraiment bien avec les images qui ont subit certaines modifications (arrière-plan complexe ou brouillé, lignes, occlusion partielle, déformation, etc.) voila pourquoi il faut faire du prétraitement sur des images avant d'extraire du texte.
3. Il peut parfois renvoyer du charabia (faux positifs).
Tesseract analyse dans l’ordre de lecture naturel des documents, ce qui n’est pas toujours la bonne méthode. Par exemple, il peut ne pas reconnaître qu’un document contient plusieurs colonnes et peut essayer de joindre le texte de ces colonnes comme une seule ligne.
4. Il a du mal à interpréter des numérisations de mauvaise qualité.
5. Il n’expose pas d’informations/métadonnées sur le texte (comme la police).
